In [1]:
import os

In [3]:
%pwd

'f:\\College\\Masters\\Semesters\\Fall 2024\\MLOps\\eCom-chat'

In [2]:
os.chdir('../')

In [18]:
REVIEW_TABLE = "ecom-chat-437005.ecom_chat.review"
META_TABLE = "ecom-chat-437005.ecom_chat.meta"
PARENT_ASIN = '0156031191'

## Connect Big Query

In [5]:
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
    'big_credentials.json')

In [6]:
from google.cloud import bigquery
client = bigquery.Client(credentials=credentials)

In [23]:
# Perform a review query.
review_query = (f'''SELECT *
         FROM {REVIEW_TABLE} 
         WHERE parent_asin='0156031191'
         ''')
query_job = client.query(review_query)  # API request
review = query_job.result()  # Waits for query to finish

In [24]:
review_df = review.to_dataframe()
review_df.head()

f:\College\Masters\Semesters\Fall 2024\MLOps\eCom-chat\ecom\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,verified_purchase,timestamp,helpful_vote,user_id,asin,parent_asin,images,rating,text,title
0,True,1421200284000,15,AGTPZ3RQ5ZN2EXMFVWJIQTYLQ4QQ,0156031191,0156031191,[],2.0,Disclaimer: I'm not a critic or a book reviewe...,750 pages of beautifully written words
1,False,1355127036000,2,AEIMMHZZUYQE7XD6KMGAPKGS5YUQ,0156031191,0156031191,[],5.0,"I think it's called ""Winter's Tale"" in part be...",A Tale that Reveals the Hidden Structure of Th...
2,True,1402007987000,21,AGXTFGO7KXIW257RRFLSA3C5VRWA,0156031191,0156031191,[],5.0,It is rare that I have read a book that I woul...,One of the greatest works of fantasy
3,True,1376062643000,34,AGVVKELAW5M3CN7TGL3MBN6SBV6A,0156031191,0156031191,[],2.0,Like a parlor magician who distracts you with ...,Not sure if it's worth finishing
4,True,1390670644000,1,AH64RBZ4TJKY57IT47YWDNDLFPNA,0156031191,0156031191,[],5.0,"My favorite book in the world!<br />Slapstick,...",Oh. My. God!


In [25]:
# Perform a metadata query.
meta_query = (f'''SELECT *
         FROM {META_TABLE} 
         WHERE parent_asin='{PARENT_ASIN}'
         ''')
query_job = client.query(meta_query)  # API request
meta = query_job.result()  # Waits for query to finish

In [26]:
meta_df = meta.to_dataframe()
meta_df.head()

f:\College\Masters\Semesters\Fall 2024\MLOps\eCom-chat\ecom\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
f:\College\Masters\Semesters\Fall 2024\MLOps\eCom-chat\ecom\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:207: UserWarning: Unable to determine Arrow type for field 'images'.
  warnings.warn(


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,Books,Winter's Tale,3.7,1901,"[A #1, New York Times, Bestseller: Mark Helpri...","[From the Back Cover, ""He creates tableaux of ...",10.99,[],[],Mark Helprin (Author),"[Video Games, PC, Games]","{'dimensions': '5.31 x 1.76 x 8 inches', 'isbn...",0156031191,[]


## Loading the Review Data

In [27]:
review_df.drop(columns=['user_id', 'images'], inplace = True)
meta_df.drop(columns=['images', 'videos', 'bought_together'], inplace = True)

In [28]:
from langchain.document_loaders import DataFrameLoader

# Create a DataFrameLoader instance
loader = DataFrameLoader(review_df) 

# Load the documents
documents = loader.load()

In [29]:
documents

[Document(metadata={'verified_purchase': True, 'timestamp': 1421200284000, 'helpful_vote': 15, 'asin': '0156031191', 'parent_asin': '0156031191', 'rating': 2.0, 'title': '750 pages of beautifully written words'}, page_content="Disclaimer: I'm not a critic or a book reviewer.  Just a guy who reads a lot.  But I do have some thoughts on  Winter's Tale.<br />750 pages of beautifully written words, sentences, paragraphs, chapters and dialogue of....of....uhh...I’m not sure.  Helprin is certainly one of the finest wordsmiths of his generation.  And he writes an off times interesting conglomeration of vignettes and characters that are loosely tied together in interaction and occurrences.  But A story, this isn’t.  Oh, it has a main character, sort of.  The hero.  But a great deal of the narrative has absolutely nothing to do with him.  I’ve seen it written that this is a love story.  A chick book it is, but a love story it isn’t.  For it to be a love story there would have to be some continu

## Create RAG Chain

In [30]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_huggingface import HuggingFaceEmbeddings

In [31]:
import os
import time
import json
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

True

In [32]:
## load the GROQ and HF API Key
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

### Metadata Summary

In [37]:
## Loading the Model
meta_llm = ChatGroq(model_name="llama-3.1-8b-instant")

In [42]:
modified_details = meta_df['details'].astype(str).str.replace('{', '[')

In [58]:
print(f'''Meta Data:
    main_category: {(meta_df.at[0,'main_category'])}
    title: {(meta_df.at[0, 'title'])}
    average_rating: {(meta_df.at[0, 'average_rating'])}
    rating_number: {(meta_df.at[0, 'rating_number'])}
    features: {(meta_df.at[0, 'features'])}
    description: {(meta_df.at[0, 'description'])}
    price: {(meta_df.at[0, 'price'])}
    store: {(meta_df.at[0, 'store'])}
    categories: {(meta_df.at[0, 'categories'])}	
    details: {(modified_details.at[0])}''')

Meta Data:
    main_category: Books
    title: Winter's Tale
    average_rating: 3.7
    rating_number: 1901
    features: ['A #1' 'New York Times'
 'Bestseller: Mark Helprin’s masterpiece transports you to New York of the Belle Epoque, to a city clarified by a siege of unprecedented snows…'
 'One winter night, Peter Lake—master mechanic and second-story man—attempts to rob a fortresslike mansion on the Upper West Side. Though he thinks it is empty, the daughter of the house is home. Thus begins the affair between a middle-aged Irish burglar and Beverly Penn, a young girl dying of consumption. It is a love so powerful that Peter, a simple and uneducated man, will be driven to stop time and bring back the dead. His great struggle is one of the most beautiful and extraordinary stories of American literature."Utterly extraordinary . . . A piercing sense of the beautiful arising from narrative and emotional fantasy is everywhere alive in the novel . . . Not for some time have I read a work

In [71]:
# Answer question
meta_system_prompt =( 
    f'''
    You are a great Data Interpreter and Summarizer. Read the Product Meta Data sent to you and Produce it in 500 words.
    
    Meta Data:
    main_category: {(meta_df.at[0,'main_category'])}
    title: {(meta_df.at[0, 'title'])}
    average_rating: {(meta_df.at[0, 'average_rating'])}
    rating_number: {(meta_df.at[0, 'rating_number'])}
    features: {(meta_df.at[0, 'features'])}
    description: {(meta_df.at[0, 'description'])}
    price: {(meta_df.at[0, 'price'])}
    store: {(meta_df.at[0, 'store'])}
    categories: {(meta_df.at[0, 'categories'])}	
    details: {(modified_details.at[0])}

    Return in a proper format:
    main_category: Same 
    title: Same
    average_rating: Same
    rating_number: Same
    features: Summarize	
    description: Summarize
    price: Same
    store: Same	
    categories: Same	
    details: Same/Summarize where necessary	
    '''
)

In [72]:
meta_system_prompt = meta_system_prompt.replace('{', '{{').replace('}', '}}')

In [73]:
meta_qa_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", meta_system_prompt),
                    ("human", "{input}")
                ]
            )

In [75]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [76]:
chain = meta_qa_prompt | meta_llm | parser

In [77]:
meta_summary = chain.invoke({"input": " "})

In [78]:
meta_summary

"**Product Meta Data Summary**\n\n**Main Category:** Books\n**Title:** Winter's Tale\n**Average Rating:** 3.7\n**Rating Number:** 1901\n\n**Features:**\n\n- A #1 New York Times Bestseller\n- A masterpiece of American literature that transports readers to New York City during the Belle Epoque\n- A poignant love story between a middle-aged Irish burglar and a young girl dying of consumption\n- A tale of timelessness and the transformative power of love\n\n**Description:**\n\nWinter's Tale is a magical and captivating novel by Mark Helprin that explores the depths of human emotion and the power of love. The story follows Peter Lake, a skilled mechanic and second-story man, as he falls deeply in love with Beverly Penn, a young girl dying of consumption. When she dies, Peter is driven to stop time and bring her back to life, embarking on a quest that takes him to the very limits of his existence.\n\n**Price:** $10.99\n**Store:** Mark Helprin (Author)\n**Categories:** ['Fiction', 'Literary F

### RAG Chain

In [79]:
## Loading the Model
llm = ChatGroq(model_name="llama-3.1-70b-versatile")

In [80]:
## Getting the embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

f:\College\Masters\Semesters\Fall 2024\MLOps\eCom-chat\ecom\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [81]:
vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

In [82]:
retriever = vectorstore.as_retriever()

In [83]:
retriever.invoke("Is the product good?")

[Document(metadata={'verified_purchase': True, 'timestamp': 1418751512000, 'helpful_vote': 0, 'asin': '0156031191', 'parent_asin': '0156031191', 'rating': 5.0, 'title': 'Five Stars'}, page_content='Arrived on time and the quality of the product was excellent'),
 Document(metadata={'verified_purchase': True, 'timestamp': 1425885256000, 'helpful_vote': 0, 'asin': '0156031191', 'parent_asin': '0156031191', 'rating': 5.0, 'title': 'Better than the movie'}, page_content='Better than the movie.'),
 Document(metadata={'verified_purchase': True, 'timestamp': 1527197651692, 'helpful_vote': 0, 'asin': '0156031191', 'parent_asin': '0156031191', 'rating': 4.0, 'title': 'Four Stars'}, page_content='Good'),
 Document(metadata={'verified_purchase': True, 'timestamp': 1425381218000, 'helpful_vote': 0, 'asin': '0156031191', 'parent_asin': '0156031191', 'rating': 5.0, 'title': 'Five Stars'}, page_content='good')]

In [87]:
system_prompt = (
    f'''
    You are Alpha. You are a sales person selling eCommerce Products.

    Here is the product information:

    {meta_summary}
    '''
    "{context}"
    '''
    Help user answer any question regarding the product. 
    Just answer the questions in brief.

    Your responses should be clear, concise, and insightful.
    '''
)

In [88]:
qa_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    ("human", "{input}")
                ]
            )

In [89]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [98]:
response = rag_chain.invoke({"input": "Is it a good gift for my friend's birthday?"})

In [99]:
response['answer']

"It could be, depending on your friend's reading preferences. If they enjoy literary fiction, fantasy, or historical fiction, they might appreciate this book. However, considering the mixed reviews and some content concerns, it's essential to know your friend's tastes and sensitivities before gifting it."